# 연습문제 2 - 쇼어 알고리즘
## 역사적 배경

컴퓨팅에서는 알고리즘이 입력 문제의 크기에 따라 얼마나 복잡해지는지를 보고 알고리즘의 성능을 측정하곤 합니다. 예를 들어, 덧셈은 더하는 숫자들의 크기에 따라 선형적으로 커지는 알고리즘을 가지고 있습니다. 몇몇 효율적인 알고리즘이라 알려진 것들도 입력의 크기에 따라 계산의 양이 _지수적으로_ 커지는데, 이는 그 알고리즘들에 대한 비교적 크지 않은 사이즈의 입력도 현재 지구상의 어떤 컴퓨터에서든 해결되기에는 너무 크다는 것을 의미합니다. 우리는 이 사실을 전적으로 확신하기 때문에 대부분의 인터넷 보안도 몇몇 문제들의 해결 불가능함에 의존하고 있습니다.

1994년에 피터 쇼어는 양자 컴퓨터에서 효율적으로 소인수분해하는 것이 가능하다는 걸 보여주었습니다.[1] 이건 매우 놀라운 발견인데, 현재 가장 효율적인 고전 소인수분해 알고리즘은 지수적으로 커지는 알고리즘들 중 하나이기 때문입니다. 사실, [RSA 암호화는](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) 충분히 큰 숫자들을 소인수분해하는 것이 현재 불가능하다는 것에 의존하고 있습니다. 현재 고전 컴퓨터에서 소인수분해하기에 너무 큰 정수들을 양자 컴퓨터에서 소인수분해하려면 몇백만개의 큐비트들과 게이트들을 필요로 하기 때문에, 현재의 양자 컴퓨터에서 성공적으로 실행되기엔 너무 큰 회로를 필요로 합니다.

그래서 Lieven M.K. Vandersypen, Matthias Steffen, Gregory Breyta, Costantino S. Yannoni, Mark H. Sherwood, 그리고 Isaac L. Chuang은 어떻게 한참 전인 2001년에 성공적으로 양자 컴퓨터에서 15를 소인수분해 할 수 있었을까요?[2]

쇼어 알고리즘을 위한 회로를 만들 때 가장 어려운 부분은 제어된 $ay \bmod N$을 계산하는 회로를 만드는 부분입니다. 이 회로를 다항 수의 게이트들로 만드는 방법은 이미 알려져 있지만, 이는 오늘날의 컴퓨터에게는 아직 너무 큽니다. 다행히도 만약 문제에 대한 부분적인 정보를 선험적으로 알 수 있다면, 우리는 약간의 '속임수'를 써 더 효율적인 회로를 만들 수 있습니다.

위에 나온 논문의 저자들은 이 회로를 그들이 사용 가능한 하드웨어에서 실행하기 위해 $7y \bmod 15$를 수행하는 아주 간단한 회로를 발견하였습니다. 이는 전체 회로를 그들의 하드웨어에서 실행될 수 있을 만큼 작게 만들었습니다. 이 연습 문제에서는 쇼어 알고리즘에서 사용될 수 있고 `ibmq_santiago`에서 실행될 수 있는 $35y \bmod N$을 수행하는 회로를 만들어볼 것입니다.

이 연습문제에서 무슨 일이 일어나는지를 이해하고 싶다면, [쇼어 알고리즘에 대한 키스킷 교과서 페이지를](https://qiskit.org/textbook/ch-algorithms/shor.html) 읽어 보면 도움이 될 것입니다. 하지만 꼭 이 페이지를 읽지 않더라도 연습 문제를 풀 수 있을 것입니다.

### 참고자료
1. Shor, Peter W. "Algorithms for quantum computation: discrete logarithms and factoring." Proceedings 35th annual symposium on foundations of computer science. Ieee, 1994.
1. Vandersypen, Lieven MK, et al. "Experimental realization of Shor's quantum factoring algorithm using nuclear magnetic resonance." Nature 414.6866 (2001): 883-887.

## 요약: 쇼어 알고리즘

어떤 게이트가 특정 상태에게 어떤 위상을 주는지 알려주는 [_양자 위상 추정 (quantum phase estimation)_](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html)이라는 알고리즘이 있습니다. 예를 들어, 양자 위상 추정 알고리즘의 입력은 $|1\rangle$ 상태와 $Z$ 게이트일 수 있습니다. 만약 $Z$ 게이트가 $|1\rangle$ 상태에 적용된다면, 전역 위상만 $\pi$만큼 추가된 같은 상태를 받을 것입니다:

$$
Z|1\rangle = -|1\rangle = e^{i\pi} |1\rangle
$$

양자 위상 추정 알고리즘은 이를 계산해줄 수 있습니다. 또 다른 예는 [여기서](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html#2.-Example:-T-gate-) 볼 수 있습니다.

쇼어는 만약 $U|y\rangle = |a y\bmod N\rangle$의 성질을 가진 어떤 게이트 $U$에 대해 위상 추정을 한다면, $N$의 인수들에 대한 정보를 단시간 안에 얻을 수 있다는 걸 보여주었습니다.

## 도전 과제

이 도전 과제에서 우리는 $13y \bmod 35$를 수행하는 회로에 대해 위상 추정을 함으로써 35를 소인수분해할 것입니다. 이 과제는 이 작업을 수행하고 `ibmq_santiago`에서 실행될 수 있을 만큼 작은 회로를 만드는 것입니다. 이는 쉬운 작업이 아니기 때문에 우리는 먼저 약간의 속임수를 써 볼 것입니다.

우리의 회로는 시작 상태 $|1\rangle$에 $U$를 적용함으로써 얻을 수 있는 상태들에만 작업하면 됩니다. 즉, 우리는 다음의 성질을 가지기만 하면, _어느_ 회로든 사용할 수 있습니다: 

$$
\begin{aligned}
U|1\rangle &= |13\rangle \\
UU|1\rangle &= |29\rangle \\
UUU|1\rangle &= |27\rangle \\
UUUU|1\rangle &= |1\rangle \\
\end{aligned}
$$

그래서 어떻게 이 작업을 더 쉽게 만들 수 있을까요? 우리는 4개의 다른 상태들만 정확하게 변환시키면 되기 때문에 이를 2개의 큐비트에 부호화시킬 수 있습니다. 이 과제에서는 2-큐비트 computational  상태들을 아래에 나온 것처럼 숫자들에 매핑시킬 것입니다:

$$
\begin{aligned}
|1\rangle &\rightarrow |00\rangle \\
|13\rangle &\rightarrow |01\rangle \\
|29\rangle &\rightarrow |10\rangle \\
|27\rangle &\rightarrow |11\rangle \\
\end{aligned}
$$

이게 왜 “속임수”일까요? 일단 이 최적화를 제대로 이용하기 위해서는 $U$가 영향을 미칠 모든 상태들을 알아야 합니다. 그러기 위해 우리는 $ay \bmod N$을 다시 1을 얻을 때까지 계산해야 하고, 그럼으로써 $a^x \bmod N$의 주기를 알게 되고 $N$의 인수들을 얻을 수 있게 될 것입니다. 이렇게 $r$ 값을 알려줄 정보를 사용하는 최적화는 고전 컴퓨터가 해결할 수 없을만큼 큰 문제가 되지는 않을 것입니다. 

하지만 이 도전 과제의 목적은 그저 쇼어 알고리즘이 작동하는 것을 확인하기만 하는 것이기 때문에, 우리가 $U$를 위한 회로를 얻기 위해 속임수를 썼다는 사실에 대해 크게 신경을 쓰진 않을 것입니다.

<div id='u-definition'></div>
<div class="alert alert-block alert-success">

**도전 과제 2a:** 다음의 변환을 수행하는 회로 ($U$)를 만드세요:

$$
\begin{aligned}
U|00\rangle &= |01\rangle \\
U|01\rangle &= |10\rangle \\
U|10\rangle &= |11\rangle \\
U|11\rangle &= |00\rangle \\
\end{aligned}
$$

또한, 이 회로는 다른 큐비트에 의해 제어되어야 합니다. 이 회로는 ‘target’이란 이름을 가진 2-큐비트 표적 레지스터에 대해 작동할 것이며, ‘control’이란 이름을 가진 다른 단일 큐비트 레지스터에 의해 제어될 것입니다. 완성된 회로는 변수 '`cu`'에 대입해야 할 것입니다.
    
</div>

In [ ]:
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, QuantumCircuit
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu = QuantumCircuit(c, t, name="Controlled 13^x mod 35")

# 이 선들 사이에 코드를 적으십시오 - 시작





# 이 선들 사이에 코드를 적으십시오 - 끝

cu.draw('mpl')

밑의 칸을 실행해 답을 확인하세요:

In [ ]:
# 다음 코드를 사용해 답을 확인하세요
from qc_grader import grade_ex2a
grade_ex2a(cu)

축하합니다! 어려운 부분을 무사히 끝내셨습니다. 

큐비트들을 측정하여 위상 추정 알고리즘의 출력을 읽기 때문에, ‘counting’ 레지스터가 $r$을 읽기 위해 충분한 수의 큐비트를 가지고 있는지 확인해야 할 것입니다. 우리의 경우에는, $r = 4$ 이기 때문에 $\log_2(4) = 2$ 수의 큐비트만 필요합니다 ($r$을 이미 알고 있기 때문에 사용하는 다른 속임수입니다). 하지만 Santiago가 5개의 큐비트를 가지고 있고, 우리는 ‘target’ 레지스터를 위해 2개의 큐비트만 사용했기 때문에 나머지 3개의 큐비트를 모두 counting 레지스터를 위해 사용할 것입니다.

$U$에 대해 위상 추정을 하기 위해서는 $n$ counting 큐비트들로 이루어진 레지스터의 (인덱스 $x$를 가진) 각 큐비트에게 ($U$를 $2^x$ 번 반복하는) $U^{2^x}$를 수행하는 회로들을 만들어야 할 것입니다. 우리의 경우에는 다음의 작업을 수행하는 세 개의 회로들이 필요합니다:

$$ U, \; U^2, \; \text{and} \; U^4 $$

따라서 다음 작업은 $U^2$를 수행하는 회로를 만드는 것입니다 ($U$를 두 번 적용하는 회로랑 같습니다).

<div class="alert alert-block alert-success">

**도전 과제 2b:** 다음의 변환을 수행하는 회로 ($U^2$)를 만드세요:

$$
\begin{aligned}
U|00\rangle &= |10\rangle \\
U|01\rangle &= |11\rangle \\
U|10\rangle &= |00\rangle \\
U|11\rangle &= |01\rangle \\
\end{aligned}
$$

또한, 이 회로는 다른 큐비트에 의해 제어되어야 합니다. 이 회로는 ‘target’이란 이름을 가진 2-큐비트 표적 레지스터에 대해 작동할 것이며, ‘control’이란 이름을 가진 다른 단일 큐비트 레지스터에 의해 제어될 것입니다. 완성된 회로는 변수 '`cu2`'에 대입해야 할 것입니다.
</div>

In [ ]:
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu2 = QuantumCircuit(c, t)

# 이 선들 사이에 코드를 적으십시오 - 시작





# 이 선들 사이에 코드를 적으십시오 - 끝

cu2.draw('mpl')

밑의 칸을 실행해 답을 확인하세요:

In [ ]:
# 다음 코드를 사용해 답을 확인하세요
from qc_grader import grade_ex2b
grade_ex2b(cu2)

마지막으로, $U$을 네 번 적용하는 것과 똑같은 회로가 필요합니다 (회로 $U^4$가 필요합니다).  

<div class="alert alert-block alert-success">
    
**도전 과제 2c:** 다음의 변환을 수행하는 회로 ($U^4$)를 만드세요:

$$
\begin{aligned}
U|00\rangle &= |00\rangle \\
U|01\rangle &= |01\rangle \\
U|10\rangle &= |10\rangle \\
U|11\rangle &= |11\rangle \\
\end{aligned}
$$

또한, 이 회로는 다른 큐비트에 의해 제어되어야 합니다. 이 회로는 ‘target’이란 이름을 가진 2-큐비트 표적 레지스터에 대해 작동할 것이며, ‘control’이란 이름을 가진 다른 단일 큐비트 레지스터에 의해 제어될 것입니다. 완성된 회로는 변수 '`cu4`'에 대입해야 할 것입니다. _힌트: 최선의 해법은 매우 간단합니다._
</div>

In [ ]:
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu4 = QuantumCircuit(c, t)

# 이 선들 사이에 코드를 적으십시오 - 시작





# 이 선들 사이에 코드를 적으십시오 - 시작

cu4.draw('mpl')

밑의 칸을 실행해 답을 확인하세요:

In [ ]:
# 다음 코드를 사용해 답을 확인하세요
from qc_grader import grade_ex2c
grade_ex2c(cu4)

<div class="alert alert-block alert-success">

**마지막 도전 과제 2:** 이제 제어된 $U$, $U^2$, 그리고 $U^4$를 얻었으니, 다같이 합쳐서 쇼어 알고리즘의 양자 부분을 구동하는 회로를 만들어 보세요.

초기화 부분은 쉽습니다: counting 레지스터를 $|{+}{+}{+}\rangle$ 상태로 두고 (세 개의 H 게이트를 사용해 할 수 있습니다) target 레지스터를 $|1\rangle$ 상태로 두면 됩니다 (이 상태는 이미 computational basis 상태 $|00\rangle$에 매핑되어 있으므로 여기서는 아무것도 하지 않아도 됩니다). 이는 모두 우리가 여러분을 위해 이미 해놓았습니다.

_여러분의_ 과제는 초기화 단계와 양자 푸리에 역변환 단계 사이에 사용될 제어된 $U$를 적용시키는 회로를 만드는 것입니다. 더 형식적으로 표현하자면, 다음의 회로를 만들면 됩니다:


$$
CU_{c_0 t}CU^2_{c_1 t}CU^4_{c_2 t}
$$

여기서 $c_0$, $c_1$, 그리고 $c_2$는 ‘counting’ 레지스터의 세 큐비트들 이고, $t$는 ‘target’ 레지스터이며, $U$는 <a href="#u-definition">과제의 첫 번째 부분에서 정의되어 있습니다</a>. 이 표기법에서 $CU_{a b}$는 $CU$가 $a$에 의해 제어되고, $b$에 대해 작용한다는 것을 의미합니다. 이에 대한 쉬운 해법은 단순히 위에서 만든 `cu`, `cu2` 그리고 `cu4` 회로들을 합치는 것이지만, 이와 같은 성질을 가진 더 효율적인 회로를 찾을 수도 있습니다!
    
</div>
<div class="alert alert-block alert-danger">
    
여러분의 회로는 오로지 [제어 반전 게이트들(CNOTs)](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html)과 단일 큐비트 [U 게이트들](https://qiskit.org/documentation/stubs/qiskit.circuit.library.UGate.html)만을 포함할 수 있습니다. 다중 큐비트 게이트가 단일 큐비트 게이트보다 하드웨어에서 실행되기 훨씬 더 어렵기 때문에, 여러분의 점수는 여러분이 사용하는 제어 반전 게이트들(CNOTs)의 숫자로 결정될 것입니다 (낮을수록 더 좋습니다). 이 조건을 만족하는데 어려움을 겪는다면, 우리가 제출 칸 안에 포함한 회로를 이 형식에 맞게 변환해주는 코드를 사용하면 됩니다. 다만, 직접 했을 때 아마 더 높은 점수를 얻을 수 있을 겁니다.
    
</div>

In [ ]:
# 이전의 답들을 합쳐 최종 제출로 만들 코드
cqr = QuantumRegister(3, 'control')
tqr = QuantumRegister(2, 'target')
cux = QuantumCircuit(cqr, tqr)
solutions = [cu, cu2, cu4]
for i in range(3):
    cux = cux.compose(solutions[i], [cqr[i], tqr[0], tqr[1]])
cux.draw('mpl')

In [ ]:
# 다음 코드를 사용해 답을 확인하세요
from qc_grader import grade_ex2_final
# 만약 회로를 제어 반전 게이트들과 단일 큐비트 게이트들로 변환하고 싶다면 아래 두 줄의 주석을 해제하세요
#from qiskit import transpile
#cux = transpile(cux, basis_gates=['cx','u'])
grade_ex2_final(cux)

만든 회로에 만족한다면, 아래에서 제출하세요:

In [ ]:
# 답을 제출하세요. 언제든 다시 제출할 수 있습니다.
from qc_grader import submit_ex2_final
submit_ex2_final(cux)

축하합니다! 도전 과제를 무사히 마쳤습니다. 여러분의 회로가 어떻게 35를 소인수분해하는 데에 쓰이는지 알고 싶다면, 아래를 계속 읽어보세요.

## 만든 회로를 사용해 35 소인수분해하기

아래의 코드 칸은 여러분이 과제에 제출한 코드를 가져와 $\tfrac{s}{r}$을 계산해줄 회로를 만듭니다. 여기서 $s$는 $0$과 $r-1$ 사이의 무작위 정수이며, $r$은 함수 $f(x) = 13^x \bmod 35$의 주기입니다.

In [ ]:
from qiskit.circuit.library import QFT
from qiskit import ClassicalRegister
# 회로 객체를 만듭니다
cr = ClassicalRegister(3)
shor_circuit = QuantumCircuit(cqr, tqr, cr)

# 큐비트들을 초기화합니다
shor_circuit.h(cqr)

# 위에서 만든 회로를 추가합니다
shor_circuit = shor_circuit.compose(cux)

# 양자 푸리에 역변환을 실행하고 출력을 추출합니다
shor_circuit.append(QFT(3, inverse=True), cqr)
shor_circuit.measure(cqr, cr)
shor_circuit.draw('mpl')

이 회로를 트랜스파일하고 얼마나 큰지, 얼마나 많은 제어 반전 게이트들을 사용하는지 확인해봅시다:

In [ ]:
from qiskit import Aer, transpile
from qiskit.visualization import plot_histogram
qasm_sim = Aer.get_backend('aer_simulator')
tqc = transpile(shor_circuit, basis_gates=['u', 'cx'], optimization_level=3)
print(f"circuit depth: {tqc.depth()}")
print(f"circuit contains {tqc.count_ops()['cx']} CNOTs")

그리고 결과를 확인해봅시다:

In [ ]:
counts = qasm_sim.run(tqc).result().get_counts()
plot_histogram(counts)

모든 게 정확하게 실행됬다고 추정하면, 각 $0$, $2$, $4$ 그리고 $8$을 측정할 확률들은 동일하게 나올 것입니다. 이는 위상 추정이 $2^n \cdot \tfrac{s}{r}$의 값을 주기 때문인데, 여기서 $n$은 counting 레지스터의 큐비트 수이며 (여기서 $n = 3$입니다), $s$는 $0$과 $r-1$ 사이의 무작위 정수이고, $r$은 우리가 계산하려는 값입니다. 이를 (고전적으로 쉽게 계산할 수 있) $s/r$을 알려주는 분수로 변환해봅시다:

In [ ]:
from fractions import Fraction
n = 3  # n은 'counting' 레지스터 속의 큐비트 수입니다
# 각 측정 문자열에 대해 반복합니다
for measurement in counts.keys():
    # 이진 문자열을 'int'로 변환하고, 2^n으로 나눕니다
    decimal = int(measurement, 2)/2**n
    # 연속 분수 알고리즘을 사용해 a/b 형식으로 변환합니다
    print(Fraction(decimal).limit_denominator())

몇몇 결과의 분모가 올바른 답 $r = 4$를 알려줄 것임을 볼 수 있습니다. $r=4$인 것을 쉽게 확인할 수 있습니다:

In [ ]:
13**4 % 35

그래서 여기서 어떻게 인수들을 얻을 수 있을까요? $N$의 최대공약수와 $a^{r/2}-1$ 아니면 $a^{r/2}+1$가 $N$의 인수일 가능성이 매우 높습니다. 그리고 최대공약수는 고전적으로 쉽게 계산할 수 있습니다.

In [ ]:
from math import gcd # 최대공약수
for x in [-1, 1]:
    print(f"Guessed factor: {gcd(13**(4//2)+x, 35)}")

하나의 인수만 찾으면 그걸로 $N$을 나누어 다른 인수를 찾을 수 있습니다. 하지만 이 경우에는, $a^{r/2}-1$와 $a^{r/2}+1$ _둘 다_ $35$의 인수를 줍니다. 이 사실도 다음과 같이 확인할 수 있습니다:

In [ ]:
7*5

## `ibmq_santiago`에서 실행하기

Santiago에서 직접 실행해보기로 했으니, 여기서는 어떻게 이를 수행할 수 있는지 보여줄 것입니다. 이 예시에서는 편리성을 위해 Santiago 장치의 시뮬레이션을 사용하였으나, 원한다면 이를 실제 장치로 바꿀 수도 있습니다:

In [ ]:
from qiskit.test.mock import FakeSantiago
from qiskit import assemble
from qiskit.visualization import plot_histogram
santiago = FakeSantiago()
real_device = False

## 실제 장치에서 실행하려면 이 코드 블록의 주석을 해제하세요
#from qiskit import IBMQ
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#santiago = provider.get_backend('ibmq_santiago')
#real_device = True

# Santiago에서 실행하기 위해 트랜스파일해야 합니다
tqc = transpile(shor_circuit, santiago, optimization_level=3)

if not real_device:
    tqc = assemble(tqc)

# 회로를 실행하고 counts 결과를 프린트합니다
counts = santiago.run(tqc).result().get_counts()
plot_histogram(counts)

만약 점수가 충분히 낮다면, 완벽한 시뮬레이션에서 보았던 것과 같이 $0$, $2$, $4$ 또는 $8$을 측정할 확률이 매우 높은 것을 확인할 수 있습니다. 이와 다른 결과들은 프로세서의 오류와 큐비트와 주변 환경의 원치 않은 상호작용 때문에 보여지는 것입니다. 이러한 '잡음'은 회로가 더 길수록 더 나빠지는데, 이는 계산 시간이 더 길수록 원치 않은 상호작용도 더 늘어나고, 더 많은 게이트는 더 많은 잠재적인 에러를 초래하기 때문입니다. 그렇기 때문에 우리는 속임수를 써 가능한 한 작은 회로를 만들어야 했습니다.

가까운 미래에는 양자 시스템들이 더 개선되어 이러한 문제들을 극복하기 위해 더 향상된 에러 완화 기술을 쓸 수 있게 될 것이며, 이는 우리가 [쇼어 알고리즘을 속임수 없이 실행할 수 있을 만큼](https://arxiv.org/pdf/quant-ph/0205095.pdf) 큰 회로를 실행할 수 있을 것을 의미합니다.

## 추가 정보

**출제자:** Frank Harkins

**번역:** Hyun Jin Kim

**버젼:** 1.0.0